##import

### import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pycaret.regression import *
from sklearn.cluster import KMeans
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
import joblib
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from pycaret import *

### 메소드

In [ ]:

# smape 계산 함수
def smape_fnc(true, pred):
  v = 2*abs(pred-true)/(abs(pred)+abs(true))
  output=np.mean(v)*100
  return output

# custom metrics
def custom_metrics(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

### 데이터 불러오기

In [ ]:
path_drive = './csv/'
path = './'

In [ ]:
combined_clust_df = pd.read_csv(path_drive+'combined_clust_df.csv')

## 군집화

### 각 군집별로 모델 생성(et) +모델별 feature 추가

Cluster 0: ['건물기타', '대학교', '연구소', '할인마트']  
Cluster 1: ['백화점및아울렛', '호텔및리조트']  
Cluster 2: ['데이터센터', '아파트']  
Cluster 3: ['공공', '병원', '상용', '지식산업센터']

In [ ]:
clust_to_num = {0: [1, 3, 9, 11], 1: [5, 12], 2: [4, 8], 3: [2, 6, 7, 10]}


#### clust0

In [ ]:
selected_columns0 = ['전력소비량(kWh)',
 '연면적(m2)',
  '연구소',
  'work_time',
  '건물번호',
  'holiday',
  'particular',
  '냉방면적(m2)',
  '시간_cos',
  '대학교',
  '불쾌지수',
  'low_day',
  '불쾌지수_이동평균3',
  '시간',
  '태양광용량(kW)',
  'ESS저장용량(kWh)',
  '월',
  '불쾌지수_이동평균5',
  '시간_sin',
  '기온(C)',
  '요일',
  '일',
 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns0]

clust0_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 0].reset_index(drop=True)



In [ ]:
# 스태킹

# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(clust0_train_df, clust0_train_df['전력소비량(kWh)'], test_size=0.2, random_state=42)

# pycaret 설정
exp0 = setup(data=X_train.drop(columns=['km_cluster']), target='전력소비량(kWh)')

# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')  # 랜덤 포레스트 모델
catboost_model = create_model('catboost')  # catboost 모델
lightgbm_model = create_model('lightgbm')  # LightGBM 모델
#dt_model = create_model('dt')  # dt 모델
#xgboost_model = create_model('xgboost')  # xgboost 모델


# 모델 리스트
models = [et_model, rf_model, catboost_model, lightgbm_model]

stacker = stack_models(estimator_list = models, meta_model = et_model)
stacker_final = finalize_model(stacker)

# 테스트 데이터 예측
y_pred = predict_model(stacker_final, data=X_test.drop(columns=['km_cluster']))

# SMAPE 계산
smape_score = smape_fnc(y_test, y_pred['prediction_label'])
print(f"SMAPE: {smape_score:.5f}")